In [ ]:
'''
Important Note:
        I have made modifications to the code in a text editor, without running it (as I don't have access
        to EMME4 anymore). so, some errors are likely to arise.
        Please contact me upon occurrence of errors of any kind.
'''

import datetime
import os 
#There is no need to import inro.modeller; it's preloaded in EMME4 Notebook.

def sortByNRI(source, demandMatrix, CPU, links2disrupt = [], disruptBothDirs = False):
    
    '''
    sortByNRI ranks links in order of their Network Robustness Index (NRI). 
    
    Requires:
    source: 
        Identifier of the original scenario wherein the intact network lies (it makes a copy of the
        original scenario to remove links from, so there's nothing to worry about.)
    demandMatrix: 
        Identifier of the demand matrix to be assigned to the network
    CPU:
        Number of CPU cores to utilize
    links2disrupt (optional): 
        Specific links to be disrupted; enter as follows -> [(i,j), (k,m), ... ]
    disruptBothDirs (optional):
        Whether to disrupt both directions in two-way roadways
    
    Returns: a pandas Series containing links ranked in order of their NRI value
    '''

    
    '''Initialization'''
    
    #in case of considerably large networks, we cannot afford to save results in DataFrames using RAM. 
    #so as to avoid "out of memory" error:
    #directories for text files to record the results in, using "with open(...)" 
    datetimeStr = str(datetime.datetime.now()).replace(':', '_')
    
    spttFileDir = datetimeStr + " spttFile.txt"
    totalTTFileDir = datetimeStr + " totalTTFile.txt
    asLinkVolFileDir = datetimeStr + " asLinkVolFile.txt"
    
    scenID = source + 1000
    _m = inro.modeller.Modeller()
    emmebank = _m.emmebank
    desktop = _m.desktop
    
    
    asLinkVol = {} #dictionary to record traffic volumes in links in different scenarios
    totalTT = {} #dictionary to record total travel time in different scenarios
    spttMat = {} #dictionary to record shortest path travel time between ODs in different scenarios
    hist = [] #links that are disrupted as the opposing direction of a previously disrupted link
    
    #ensuring that there is no attribute with identifier "@alvol" created previously:
    try:
        _m.tool("inro.emme.data.extra_attribute.delete_extra_attribute")("@alvol") 
    except:
        pass
    
    #creating an attribute to save link volumes after assignment:
    alvol = _m.tool("inro.emme.data.extra_attribute.create_extra_attribute")(extra_attribute_type="LINK",
                        extra_attribute_name = "@alvol",
                        extra_attribute_description = "link volumes after assignment",
                        overwrite = True) 
    
    #ensuring that there is no matrix with identifier "mf30" created previously:
    try: 
        _m.tool("inro.emme.data.matrix.delete_matrix")("mf30")
    except:
        pass

    #creating a matrix to save OD travel times on shortest paths:
    sptt = _m.tool("inro.emme.data.matrix.create_matrix")(matrix_id="mf30",
                            matrix_name = "sptt",
                            matrix_description = "O-D travel times on shortest paths",
                            default_value = 0) 
    
    demandNumpyMatrix = emmebank.matrix(demandMatrix).get_data().to_numpy() #for matrix multiplication
    
    #ensuring that there is no scenario with identifier scenID created previously:
    try: 
        emmebank.delete_scenario(scenID)
    except: 
        pass
    
    #a copy of the original scenario to disrupt links in:
    emmebank.copy_scenario(source,scenID)
    scen = emmebank.scenario(scenID)
    
    #specifying the assignment tool (in JSON):
    assign = _m.tool('inro.emme.traffic_assignment.standard_traffic_assignment')
    null = None
    specs = '''{
        "type": "STANDARD_TRAFFIC_ASSIGNMENT",
        "classes": [
            {
                "mode": "c",
                "demand": "''' + demandMatrix + '''" ,
                "generalized_cost": null,
                "results": {
                    "link_volumes": "@alvol",
                    "turn_volumes": null,
                    "od_travel_times": {
                        "shortest_paths": "mf30"
                    }
                },
                "analysis": {
                    "analyzed_demand": null,
                    "results": {
                        "od_values": null,
                        "selected_link_volumes": null,
                        "selected_turn_volumes": null
                    }
                }
            }
        ],
        "performance_settings": {
            "number_of_processors": ''' + str(CPU) + '''
        },
        "background_traffic": null,
        "path_analysis": null,
        "cutoff_analysis": null,
        "traversal_analysis": null,
        "stopping_criteria": {
            "max_iterations": 100,
            "relative_gap": 0,
            "best_relative_gap": 0.1,
            "normalized_gap": 0.05
        }
    }'''
    
    '''traffic assignment to the base scenario'''
    assign(specification = specs, scenario = scen)
    #recording the results:
    alvol = scen.get_attribute_values("LINK",["@alvol"])
    linkVol = {} #dictionary to record traffic volumes in links
    for i in alvol[0]:
        for j in alvol[0][i]:
            linkVol[(i,j)] = alvol[1][alvol[0][i][j]]
    
    asLinkVol['base'] = linkVol
    totalTT['base'] = (sptt.get_data().to_numpy() * demandNumpyMatrix).sum()
    spttMat['base'] =  sptt.get_data()
    
    with open(spttFileDir, 'a+') as f:
        f.write('{"base" : ' + str(sptt.get_data()))
    with open(totalTTFile, 'a+') as f:
        f.write('{"base" : ' + str((sptt.get_data().to_numpy() * demandNumpyMatrix).sum()))
    with open(asLinkVolFile, 'a+') as f:
        f.write('{"base" : ' + str(linkVol))
    
    '''disrupt, assign, reload the intact network'''
    with inro.modeller.logbook_trace("Sequential Assignment"):
        
        #if the user hasn't inputted any specific links to disrupt, go through the whole network.
        if links2disrupt == []: 
            links = scen.get_network().links()
        else:
            links = [scen.get_network().link(i,j) for i,j in links2disrupt]
        
        for link in links:
            if disruptBothDirs:
                #skip link if it has been already disrupted as an opposing direction:
                if link in hist:
                        continue
                #if link belongs to a two-way roadway, append the opposing direction to hist:
                if link.reverse_link != None: 
                    linkDirs = [link , link.reverse_link]
                    hist.append(link.reverse_link) 
                    twoWay = True
            else:
                linkDirs = [link]
                twoWay = False

            #disrupting link(s):
            for linkDir in linkDirs: 
                i_node = linkDir.i_node
                j_node = linkDir.j_node
                _m.tool("inro.emme.data.network.base.delete_links")(selection="link=" + str(i_node) + ',' 
                                                                    + str(j_node),condition="cascade", 
                                                                    scenario=scen)
            #assigning traffic to the modified network:
            assign(specification = specs, scenario = scen)
            #recording the results:
            alvol = scen.get_attribute_values("LINK",["@alvol"])
            linkVol = {}
            for i in alvol[0]:
                for j in alvol[0][i]:
                    linkVol[(i,j)] = alvol[1][alvol[0][i][j]]

            asLinkVol[(int(i_node.id),int(j_node.id))] = linkVol
            asLinkVol[(int(j_node.id),int(i_node.id))] = linkVol
            
            totalTT[(int(i_node.id),int(j_node.id))] = (sptt.get_data().to_numpy() 
                                                        * demandNumpyMatrix).sum() 
            totalTT[(int(j_node.id),int(i_node.id))] = (sptt.get_data().to_numpy() 
                                                        * demandNumpyMatrix).sum()

            spttMat[(int(i_node.id),int(j_node.id))] = sptt.get_data()
            spttMat[(int(j_node.id),int(i_node.id))] = sptt.get_data()
            
            with open(spttFileDir, 'a+') as f:
                f.write(', "' + str((int(i_node.id),int(j_node.id))) + '" : ' + str(sptt.get_data()))
            with open(totalTTFile, 'a+') as f:
                f.write(', "' + str((int(i_node.id),int(j_node.id))) + '" : ' 
                        + str((sptt.get_data().to_numpy() * demandNumpyMatrix).sum()))
            with open(asLinkVolFile, 'a+') as f:
                f.write(', "' + str((int(i_node.id),int(j_node.id))) + '" : ' + str(linkVol))
            
            if disruptBothDirs:
                if twoWay:
                    with open(spttFileDir, 'a+') as f:
                        f.write(', "' + str((int(j_node.id),int(i_node.id)))+'" : ' 
                                + str(sptt.get_data()))
                    with open(totalTTFile, 'a+') as f:
                        f.write(', "' + str((int(j_node.id),int(i_node.id)))+'" : '
                                + str((sptt.get_data().to_numpy() * demandNumpyMatrix).sum()))
                    with open(asLinkVolFile, 'a+') as f:
                        f.write(', "' + str((int(j_node.id),int(i_node.id)))+'" : '+str(linkVol))
                
            emmebank.delete_scenario(scenID)
            emmebank.copy_scenario(source,scenID)
    
    with open(spttFileDir, 'a+') as f:
        f.write('}')
    with open(asLinkVolFile, 'a+') as f:
        f.write('}')
    with open(totalTTFile, 'a+') as f:
        f.write('}')
    
    #calculation of NRI:
    with open(totalTTFile, 'r+') as f: 
        totalTT_Series = pd.Series(eval(f))
    NRI_Series = totalTT_Series.subtract(totalTT_Series['base'])
    NRI_Series.drop('base', inplace = True)
    NRI_Series.sort_values(ascending = False, inplace = True)
    
    print ('Detailed result files are saved in ' + os.getcwd())
    
    return NRI_Series
